In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
from experiments.iter2.util.importer_LSTM import *
GRAPH_PATH = 'graph_saves/ex13_graphs/ex13_'
MODEL_PATH = 'model_saves/ex13_models/ex13_model'
nist_path = "../../src/data_preprocess/dataset/NIST_cleaned.csv"
SEED = 1337
SEED_COUNT = 15
from experiments.iter1.util.evaluate import evaluate_model


In [ ]:
if not os.path.exists('model_saves/ex13_models'):
    os.mkdir('model_saves/ex13_models')
if not os.path.exists('graph_saves/ex13_graphs'):
    os.mkdir('graph_saves/ex13_graphs')


for i in range(SEED_COUNT):
    seed_everything(SEED, workers=True)

    assert time_horizon > 0, "Time horizon must be a positive integer"
        
    df = pd.read_csv(nist_path)

    cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
    splitter = DaySplitter(TIMESTAMP, POWER ,train_days, val_days, test_days)
        
    model = LSTM(hidden_size, num_layers, input_size, time_horizon, dropout)
    trainer = TrainerWrapper(L.Trainer, 
                            max_epochs=num_epochs, 
                            callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3, verbose=False, mode='min', strict=True)], 
                            gradient_clip_val=gradient_clipping, deterministic=True)
    optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

    model = DeterministicPipeline.Builder() \
        .add_data(df) \
        .set_cleaner(cleaner) \
        .set_normalizer_class(MinMaxNormalizer) \
        .set_splitter(splitter) \
        .set_sequencer_class(AllTimeSequencer) \
        .set_target_column(TARGET_COLUMN) \
        .set_model(model) \
        .set_optimizer(optimizer) \
        .set_batch_size(batch_size) \
        .set_seq_len(seq_len) \
        .set_worker_num(NUM_WORKERS) \
        .set_error(NRMSE) \
        .set_train_error(RMSE) \
        .set_trainer(trainer) \
        .set_use_tuner(True) \
        .build()

    model = EnsemblePipeline.Builder() \
            .set_pipeline(model) \
            .set_num_ensembles(num_ensembles) \
            .add_test_error(NRMSE) \
            .add_test_error(NMLSCV) \
            .add_test_error(NMCRPS) \
            .add_test_error(CALE) \
            .set_horizon_len(time_horizon) \
            .set_base_seed(SEED) \
            .build()

    model.fit()
    ensemble_val_loss_arr = model.get_validation_loss()
    ensemble_train_loss_arr = model.get_training_loss()
    for loss_arr in ensemble_val_loss_arr:
        plot_loss([ensemble_val_loss_arr[0], ensemble_train_loss_arr[0]], GRAPH_PATH + "LOSS_" + str(SEED), ["Validation loss", "Training loss"])
    model.save(MODEL_PATH + '_' + str(SEED))
    SEED += 1

In [ ]:
MODEL_PATH = MODEL_PATH + '_seed1'

df = pd.read_csv(nist_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = DaySplitter(TIMESTAMP, POWER ,train_days, val_days, test_days)
    
model = LSTM(hidden_size, num_layers, input_size, time_horizon, dropout)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3, verbose=False, mode='min', strict=True)], 
                         gradient_clip_val=gradient_clipping, deterministic=True)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = DeterministicPipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(AllTimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_model(model) \
    .set_optimizer(optimizer) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .set_train_error(RMSE) \
    .set_trainer(trainer) \
    .set_use_tuner(True) \
    .build()

model = EnsemblePipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(AllTimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_pipeline(model) \
    .set_num_ensembles(num_ensembles) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .add_test_error(NMLSCV) \
    .add_test_error(NMCRPS) \
    .add_test_error(CALE) \
    .set_horizon_len(time_horizon) \
    .set_base_seed(1337) \
    .build()

model.load(MODEL_PATH)
model.test()

evaluate_model(model, df, splitter, cleaner, TIMESTAMP, POWER, on_limit_w, off_limit_w, consecutive_points, seq_len, time_horizon, TARGET_COLUMN, error, temp_boundary, 0.95)